In [1]:
from dorna2 import Dorna
from camera import Camera
from dorna_vision import Detection
import tools
import config
import time
import numpy as np
import math

#####################################################################
# Connect to the robot:
robot = Dorna()
robot_connected = False
connection_try_counter = 0

while not robot_connected:
    if robot.connect(config.robot["ip"]):
        print("Robot connected")
        robot_connected = True
    else:
        print("Robot connection failed")

    if not robot_connected:
        connection_try_counter += 1
        time.sleep(1)
        if connection_try_counter == 100:
            print("Maximum connection attempts reached")

# Connect and initialize the camera
camera = Camera()
try:
    camera.connect()
except Exception as e:
    print(f"Camera connection failed: {e}")
else:
    print("Camera connected")

ROI_corners = [[226, 18], [624, 18], [624, 470], [226, 470]]
Ind_ROI_corners = [[409, 103], [554, 225], [432, 378], [283, 256]]

# Detections:
preset_general = {'roi': {'corners': ROI_corners, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': True, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial_inner.pkl', 'conf': 0.7, 'cls': []}}
detection_general = Detection(camera=camera, robot=robot, **preset_general)

# preset_individual = {'roi': {'corners': Ind_ROI_corners, 'inv': False, 'crop': True, 'offset': 0}, "camera_mount":{"type": "dorna_ta_j4","ej": [0 ,0, 0, 0, 0, 0, 0, 0],"T": [48.05923462271373, 30.4362801279415, -5.585734144443932, 0.4111487682507738, -0.18114590496686764, 90.05010215826098]}, 'display': {'label': 0, 'save_img': False, 'save_img_roi': True}, 'detection': {'cmd': 'od', 'path': 'auto_sampler_vial.pkl', 'conf': 0.8, 'cls': []}}
# detection_individual = Detection(camera=camera, robot=robot, **preset_individual)

#Go to initial position:
# tools.safe_init(config=config, robot=robot)
tools.output(config.robot_gripper["open"], robot)
tools.output(config.decapper["open"], robot)
tools.take_init_picture(config=config, robot=robot) #This should be included in the safe init motion
print("Joints before picture: ", robot.get_all_joint())
print("Target joints: ", config.camera["joints"])

# Call the detection
retval = detection_general.run()
print("Vials detected: ", len(retval))

camera_data = detection_general.get_camera_data()

# Initialize control variables
# Calculate the position of the robot's x offset in the world frame
tools.update_robot_frame(config=config)

#Generate target coordinates:
Caps_coords = tools.generate_caps_coords() # This function outputs Vial and Caps coords. After vision was introduced, these Vial coords are no longer needed

vials_coords_vision = []
vial_vis_pxls = []

for x in retval:
    # Correct xyz:
    detection_xyz = x["xyz"]
    pxl = x["center"]
    vial_vis_pxls.append(pxl)

    xyz = tools.correct_vision_xyz(config=config, pxl=pxl, camera=camera, camera_data=camera_data, detection=detection_general)
    xyz_world = tools.translate_vis_to_world(config=config, xyz=xyz)
    vials_coords_vision.append(xyz_world)

print("Vial world coords from vision: ")
for x in vials_coords_vision: print(x)
print("Vial pxls from vision: ")
for x in vial_vis_pxls: print(x)

coords_from_grid = tools.generate_vial_coords_vision(config=config, vial_vis_pxls=vial_vis_pxls)
print("Interpolated world coords from vision with griddata: ")
for x in coords_from_grid: print(x)

sorted_vials = tools.sort_vials(config=config, points=coords_from_grid)

print("Sorted world coords from vision with griddata: ")
for x in sorted_vials: print(x)

vials_to_do = len(sorted_vials)
print("Vials to do: ", vials_to_do)
ind = 0

print("Initialization completed")

init_drop_rail = 450
ref_drop_world = 771.95 # Taken from a real run, could also be: 770.19
rail_diff = ref_drop_world - init_drop_rail

camera.close()

for i in range(vials_to_do):
    if i < ind:
        continue
    elif i > ind:
        ind = i

    start_time = time.time()
    tools.update_cap_frame(config=config, cap_frame=Caps_coords[ind], rail_x=435)

    xyz_to_use = sorted_vials[ind]
    new_rail = xyz_to_use[0] - rail_diff

    tools.update_vial_frame_vision(config=config, vial_frame_vision=xyz_to_use, rail_x=new_rail)

    #Sequence:
    tools.vial_tray_to_decapper(config=config, robot=robot)
    tools.cap_to_vial(config=config, robot=robot)
    tools.capping(config=config, robot=robot)
    tools.vial_to_tray(config=config, robot=robot)
    end_time = time.time()

    print("Vial processing completed")
    print("Pick and place movement execution time: ", end_time-start_time)
    # break #To test a single vial

print("Sequence completed")

Robot connected
Camera connected
Joints before picture:  [-64.907227, 6.569824, -109.160156, 0.021973, 12.590332, -64.995117, 435.0375, 0.0]
Target joints:  [-64.907227, 6.569824, -109.138184, 0.021973, 12.546387, -64.929199]
Vials detected:  3
Vial world coords from vision: 
[772.2621178976814, -301.23929492631424, -52.42298873071273]
[773.103396001001, -286.0316230211526, -52.410507042971844]
[773.1794170027212, -316.3488155294101, -52.42317449808944]
Vial pxls from vision: 
[375, 91]
[353, 132]
[392, 48]
Interpolated world coords from vision with griddata: 
[770.5340416469616, -299.27876859858196]
[771.1952935874804, -285.11320554022865]
[771.561045089483, -313.3208191565584]
Found 1 rows
Found 3 columns
Sorted world coords from vision with griddata: 
[771.1952935874804, -285.11320554022865]
[770.5340416469616, -299.27876859858196]
[771.561045089483, -313.3208191565584]
Vials to do:  3
Initialization completed
Cap in world position updated
Vial in world position updated
Vial process

KeyboardInterrupt: 

In [ ]:
def generate_vial_drop(config, final_loc, tilt_angle, alignment_angle=45):
    init_rot = init_pose[3:]
    rvec_first = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,1,0], angle=-tilt_angle, local=False) #Rotation to insert vial
    rvec_second = robot.kinematic.rotate_rvec(rvec=rvec_first, axis=[1,0,0], angle=-tilt_angle, local=False) #Rotation to insert vial
    rvec_final = robot.kinematic.rotate_rvec(rvec=rvec_second, axis=[0,0,1], angle=-alignment_angle, local=False) #Rotation to avoid other vials
    config.vial_holder["drop"][3:] = rvec_final
    print(config.vial_holder["drop"])

init_pose = [178.653739, -217.879284, 50.71865, 180, 0, 0]
init_rot = init_pose[3:]

tilt_angle = 45
pour_rvec_first = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,1,0], angle=-tilt_angle, local=False) #Rotation to pout the water
pour_rvec_second = robot.kinematic.rotate_rvec(rvec=pour_rvec_first, axis=[1,0,0], angle=-tilt_angle, local=False) #Rotation to pout the water
pour_rvec_final = robot.kinematic.rotate_rvec(rvec=pour_rvec_second, axis=[0,0,1], angle=-45, local=False) #Included to aligned bottle pouring to gripper front

# pour_rvec_final = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[0,0,1], angle=-45, local=False)

print("Bottle pour angle updated")

print(init_pose)
print(init_rot)
print(pour_rvec_final)

final_loc = [1.2848547803481551, -0.6376452704425901, -14.334862947201302, -1.2390190108418174e-06, 1.5174821696505201e-06, 45.000000000000014]
final_orient = final_loc[3:]

[0, tilt_angle, 0]

init_rot = [180, 0, 0]

tilt_angle2 = 45
fist_rot = robot.kinematic.rotate_rvec(rvec=init_rot, axis=[1,0,0], angle=final_orient[0], local=True)
second_rot = robot.kinematic.rotate_rvec(rvec=fist_rot, axis=[0,1,0], angle=final_orient[1], local=True)
third_rot = robot.kinematic.rotate_rvec(rvec=second_rot, axis=[0,0,1], angle=final_orient[2], local=True)
fourth_rot = robot.kinematic.rotate_rvec(rvec=third_rot, axis=[0,1,0], angle=tilt_angle2, local=True)

print(third_rot)
print(fourth_rot)



Bottle pour angle updated
[178.653739, -217.879284, 50.71865, 180, 0, 0]
[180, 0, 0]
[119.01645745459449, -26.962097726556486, 26.962097726556443]
[166.29831699962298, -68.88301505518544, 2.682209014892578e-06]
[140.78179362364665, -58.31372589587072, 58.313730914363]
